<a href="https://colab.research.google.com/github/RenanNB360/Modelo-Transformer-Sem-Uso-de-Frameworks/blob/main/Implementa%C3%A7%C3%A3o_Mecanismo_de_Aten%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalações

In [ ]:
!pip install -q -U watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.7 MB/s eta 0:00:00


In [ ]:
%reload_ext watermark
%watermark -a "Attention"

Author: Attention



In [ ]:
%watermark -v -m

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 7.34.0

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 5.15.120+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



# Anatomia da Arquitetura Transformer (Com Uso do framework PyTorch)

In [ ]:
import torch
from torch import nn
print(torch.__version__)

2.1.0+cu118


In [ ]:
class Transformer(nn.Module):

  def __init__(self, vocab_size, embedding_dim, n_heads, n_layers, dropout):

    super().__init__()

    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.n_heads = n_heads
    self.n_layers = n_layers
    self.dropout = dropout

    self.embedding = nn.Embedding(vocab_size, embedding_dim)

    self.attention = nn.MultiheadAttention(embedding_dim, n_heads, dropout = dropout)

    self.feed_forward = nn.Sequential(
        nn.Linear(embedding_dim, embedding_dim),
        nn.ReLU(),
        nn.Linear(embedding_dim, embedding_dim)
    )

    self.out = nn.Linear(embedding_dim, vocab_size)

  def forward(self, x):
    x = self.embedding(x)
    x = self.attention(x)
    x = self.feed_forward(x)
    x = self.out(x)

    return x

In [ ]:
modelo = Transformer(vocab_size = 1000,
                     embedding_dim = 32,
                     n_heads = 4,
                     n_layers = 2,
                     dropout = 0.5)

In [ ]:
modelo.modules

<bound method Module.modules of Transformer(
  (embedding): Embedding(1000, 32)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
  )
  (feed_forward): Sequential(
    (0): Linear(in_features=32, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
  )
  (out): Linear(in_features=32, out_features=1000, bias=True)
)>

In [ ]:
modelo.attention

MultiheadAttention(
  (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
)

# Construindo um modelo Transformer (Sem uso de Framework)

O objetivo é que ele seja capaz de prever sequências de comprimeto igual a 10 tokens.

Um modelo transformer consiste em várias partes:



1.   Camada Embedding: Transforma as palavras em valores númericos de tamanho fixo.
2.   Mecanismo de Atenção: Permite que o modelo foque em diferentes partes da entrada.
3.   Camadas Encoder e Decoder: Processam os dados sequencialmente.
4.   Camada linear e Softmax: Para as predições finais.





Para este teste o objetivo é implementar o item 2, mas paraa deixar funcional vou implementar também os itens 1 e 4.

**Hiperparâmetros Iniciais**

In [ ]:
import numpy as np

In [ ]:
# Dimensão modelo
dim_model = 64

In [ ]:
# Comprimento da sequência
seq_length = 10

In [ ]:
# Tamanho do vocabulário
vocab_size = 100

**Camada Embedding**

In [ ]:
def embedding(input, vocab_size, dim_model):
  embed = np.random.randn(vocab_size, dim_model)
  return np.array([embed[i] for i in input])

**Função de Ativação Softmax**

In [ ]:
def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum(axis=-1).reshape(-1, 1)

**Scale Dot Product**

In [ ]:
def scaled_dot_product_attention(Q, K, V):
  matmul_qk = np.dot(Q, K.T)
  depth = K.shape[-1]
  logits = matmul_qk / np.sqrt(depth)
  attention_weights = softmax(logits)
  output = np.dot(attention_weights, V)
  return output

**Saída do Modelo com Operação Linear e Softmax**

In [ ]:
def linear_and_softmax(input):
  weights = np.random.randn(dim_model, vocab_size)
  logits = np.dot(input, weights)
  return softmax(logits)

**Construindo o Modelo Final**

In [ ]:
def transformer_model(input):
  embedded_input = embedding(input, vocab_size, dim_model)
  attention_output = scaled_dot_product_attention(embedded_input, embedded_input, embedded_input)
  output_probabilities = linear_and_softmax(attention_output)
  output_indices = np.argmax(output_probabilities, axis = 1)
  return output_indices

**Usando o Modelo para Previsões**

In [ ]:
input_sequence = np.random.randint(0, vocab_size, seq_length)

In [ ]:
print(f"Sequência de Entradas: {input_sequence}")

Sequência de Entradas: [22 30 91 79  7 67  6 20 93 40]


In [ ]:
output = transformer_model(input_sequence)

In [ ]:
print(f"Saída do Modelo: {output}")

Saída do Modelo: [87 40 34 71 99 34 93 83 91 42]
